# Evaluación con métrica no basada en un LLM

In [149]:
from ragas.metrics import DataCompyScore
from ragas.dataset_schema import SingleTurnSample

data1 = """titulo,fecha_ano,autor_a
Paseo a orillas del mar,1905,Joaquín Sorolla
Triste herencia,1899,Joaquín Sorolla
"""

data2 = """titulo,fecha_ano,autor_a
Paseo a orillas del mar,1905,Joaquín Sorolla
Niños en la playa,1905,Joaquín Sorolla y Bastida
"""

sample = SingleTurnSample(response=data1, reference=data2)
scorer = DataCompyScore()
await scorer.single_turn_ascore(sample)

np.float64(0.5)

In [150]:
from ragas.metrics import DataCompyScore
from ragas.dataset_schema import SingleTurnSample

data1 = """titulo,fecha_ano,autor_a
Paseo a orillas del mar,1905,Joaquín Sorolla
Triste herencia,1899,Joaquín Sorolla
"""

data2 = """titulo,fecha_ano,autor_a
Paseo a orillas del mar,1905,Joaquín Sorolla
Niños en la playa,1905,Joaquín Sorolla
"""

sample = SingleTurnSample(response=data1, reference=data2)
scorer = DataCompyScore(mode="columns", metric="recall")
await scorer.single_turn_ascore(sample)

0.3333333333333333

## Generar las preguntas y las consultas

In [151]:
import psycopg2

# Configuración de conexión (ajusta esto a tus credenciales)
conn = psycopg2.connect(
    host="localhost",
    database="museo",
    user="fedecl",
    password="password",
    port=5432 
)

preguntas = [
    "¿Cuántos cuadros hay en la colección?",
    "¿Qué pinturas fueron creadas en 1905?",
    "¿Cuántas esculturas hay en el museo?",
    "Dame las obras de Sorolla con temática marina.",
    "¿Qué autores tienen más de 5 obras en la colección?",
    "¿Cuántas obras hay por colección?",
    "¿Qué obras están sin fecha conocida?",
    "¿Cuáles son las obras más antiguas del museo?",
    "¿Cuántas pinturas hay con técnica al óleo?",
    "¿Qué obras fueron donadas?",
    "¿Qué obras están firmadas por Sorolla?",
    "¿Qué representa el cuadro Madre?",
    "¿Qué obras fueron producidas en Valencia?",
    "Háblame de pinturas que mencionen Jávea.",
    "Hábalame de cartas sobre la hemiplejía de Sorolla.",
]

consultas_reference = [
    "SELECT COUNT(*) FROM fichas_raw WHERE coleccion ILIKE '%pintura%'",
    "SELECT * FROM fichas_raw WHERE coleccion ILIKE '%pintura%' AND fecha_ano = 1905 LIMIT 10",
    "SELECT COUNT(*) FROM fichas_raw WHERE coleccion ILIKE '%escultura%'",
    "SELECT * FROM fichas_raw WHERE autor_a ILIKE '%sorolla%' AND (descripcion ILIKE '%mar%' OR clasificacion_razonada ILIKE '%mar%' OR historia_del_objeto ILIKE '%mar%') LIMIT 10",
    "SELECT autor_a, COUNT(*) FROM fichas_raw GROUP BY autor_a HAVING COUNT(*) > 5",
    "SELECT coleccion, COUNT(*) FROM fichas_raw GROUP BY coleccion",
    "SELECT * FROM fichas_raw WHERE fecha_ano IS NULL LIMIT 10",
    "SELECT * FROM fichas_raw WHERE fecha_ano IS NOT NULL ORDER BY fecha_ano ASC LIMIT 10",
    "SELECT COUNT(*) FROM fichas_raw WHERE coleccion ILIKE '%pintura%' AND tecnica ILIKE '%oleo%'",
    "SELECT * FROM fichas_raw WHERE forma_de_ingreso ILIKE '%donacion%' LIMIT 10",
    "SELECT * FROM fichas_raw WHERE firmas_marcas_etiquetas ILIKE '%sorolla%' LIMIT 10",
    "SELECT * FROM fichas_raw WHERE titulo ILIKE '%madre%' LIMIT 10",
    "SELECT * FROM fichas_raw WHERE lugar_de_produccion_ceca ILIKE '%valencia%' LIMIT 10",
    "SELECT * FROM fichas_raw WHERE coleccion ILIKE '%pintura%' AND (descripcion ILIKE '%javea%' OR clasificacion_razonada ILIKE '%javea%' OR historia_del_objeto ILIKE '%javea%') LIMIT 10",
    "SELECT * FROM fichas_raw WHERE coleccion ILIKE '%carta%' AND (descripcion ILIKE '%hemiplejia%' OR clasificacion_razonada ILIKE '%hemiplejia%' OR historia_del_objeto ILIKE '%hemiplejia%') LIMIT 10",
]

# Lista para guardar los resultados
resultados_reference = []

# Ejecutar consultas
cur = conn.cursor()

for consulta in consultas_reference:
    cur.execute(consulta)
    resultado = cur.fetchall()
    resultados_reference.append(resultado)

# Cerrar conexión
cur.close()
conn.close()

# Mostrar resultados combinados
for pregunta, resultado in zip(preguntas, resultados_reference):
    print(f"{pregunta} → {resultado}")


¿Cuántos cuadros hay en la colección? → [(1358,)]
¿Qué pinturas fueron creadas en 1905? → [('00330', 'pintura', None, None, 'javea', 'reverso sobre el carton compartido con la obra no inv 331 tinta impresa azul 1929 ca 73 n hace referencia a la serie y numero de la testamentaria de 1929 reverso sobre el carton compartido con la obra no inv 331 escritura manual castellano 1929 ca vda de s realizado a tinta negra completa la inscripcion a tinta impresa azul con la serie y numero de la testamentaria de 1929 indicando la adjudicacion a clotilde garcia del castillo reverso etiqueta blanca sobre el carton compartido con la obra no inv 331 tinta impresa castellanoserie n 73 realizado a tinta negra la cifra escrita entre parentesis esta escrita con tinta impresa y no manual como el resto reverso sobre el carton compartido con la obra no inv 331 sello humedo castellano 1982 ca museo sorolla 330 sello a tinta impresa azul y numero a tinta negra manual realizado tras la adjudicacion del numero de

## Generar la consulta con el LLM

In [83]:
def llamar_llm_sql(client, llm_modelname, mensaje, contexto=""):
    prompt_sql = f"""
        Genera una consulta SQL para responder la siguiente pregunta del usuario, usando la tabla `fichas_raw`, que tiene las siguientes columnas:
        - inventario
        - coleccion (mobiliario, cartas, escultura, textiles, pintura, fotografia, dibujo, joyeria, ceramica)
        - contexto_cultural_estilo
        - dimensiones
        - iconografia
        - historia_del_objeto
        - lugar_de_produccion_ceca
        - componentes
        - tecnica
        - conjunto
        - titulo
        - autor_a
        - bibliografia
        - descripcion
        - lugar_de_procedencia
        - nombre_especifico
        - clasificacion_razonada
        - materia_soporte
        - imagenes
        - forma_de_ingreso
        - firmas_marcas_etiquetas
        - datacion (datación aproximada)
        - fecha_ano (año de datación)
        - inscripciones_leyendas
        - objeto_documento
        - clasificacion_generica

        Ten en cuenta lo siguiente:
        - La base de datos contiene texto en minúsculas y sin tildes. Usa `ILIKE` con operador % para encontrar coincidencias aproximadas.
        - Para búsquedas temáticas o de contenido, es mucho más probable que las palabras clave relevantes estén en las columnas `descripcion`, `clasificacion_razonada` y `historia_del_objeto`, incluso si hay otras columnas como `lugar_de_produccion_ceca` o `tecnica` que parezcan relevantes pero no siempre están rellenas. **Prioriza siempre estos campos largos para búsquedas por palabras clave.**
        - Si se menciona un número que no parece una fecha, probablemente se refiere al `inventario`.
        - Puedes usar `ORDER BY` si la pregunta lo pide, y si no, devuelve los resultados en orden aleatorio.
        - A menos que el usuario especifique lo contrario, limita los resultados a 10 filas.
        - usa PRIMERO GROUP BY Y DESPÚES LIMIT
        - No uses instrucciones como `DELETE`, `UPDATE`, `INSERT`, `DROP` o `CREATE`.
        - Usa solo las columnas proporcionadas.

        Contexto anterior conversación (opcional): {contexto}
        Pregunta del usuario: {mensaje}

        Devuelve únicamente el texto de la consulta SQL sin comentarios ni formato adicional ni ```sql.
        """


    completion = client.chat.completions.create(
        model=llm_modelname,
        messages=[{"role": "user", "content": prompt_sql}],
        temperature=0.0,
        max_completion_tokens=512,
        top_p=1,
        stream=True
    )

    # Obtener la respuesta de la consulta SQL y comprobar que es válida (solo SELECT)   
    sql_respuesta = ""
    for chunk in completion:
        sql_respuesta += chunk.choices[0].delta.content or ""

    if not sql_respuesta.lower().startswith("select"):
        raise ValueError("La consulta SQL generada no es válida. Debe comenzar con 'SELECT'.")

    return sql_respuesta
    

In [ ]:
from groq import Groq

# Configuración de conexión (ajusta esto a tus credenciales)
conn = psycopg2.connect(
    host="localhost",
    database="museo",
    user="fedecl",
    password="password",
    port=5432 
)

client = Groq(api_key) #completar

consultas_response = []


for pregunta in preguntas:
    consulta = llamar_llm_sql(client, "llama-3.3-70b-versatile", pregunta)
    consultas_response.append(consulta)

cur = conn.cursor()

resultados_response = []

for consulta in consultas_reference:
    cur.execute(consulta)
    resultado = cur.fetchall()
    resultados_response.append(resultado)

# Cerrar conexión
cur.close()
conn.close()



In [85]:
for i in range(len(preguntas)):
    print(resultados_response[i])
    print(type(resultados_response[i]))
    print(resultados_reference[i])
    print(type(resultados_reference[i]))
    print('-'*50)

[(1358,)]
<class 'list'>
[(1358,)]
<class 'list'>
--------------------------------------------------
[('00330', 'pintura', None, None, 'javea', 'reverso sobre el carton compartido con la obra no inv 331 tinta impresa azul 1929 ca 73 n hace referencia a la serie y numero de la testamentaria de 1929 reverso sobre el carton compartido con la obra no inv 331 escritura manual castellano 1929 ca vda de s realizado a tinta negra completa la inscripcion a tinta impresa azul con la serie y numero de la testamentaria de 1929 indicando la adjudicacion a clotilde garcia del castillo reverso etiqueta blanca sobre el carton compartido con la obra no inv 331 tinta impresa castellanoserie n 73 realizado a tinta negra la cifra escrita entre parentesis esta escrita con tinta impresa y no manual como el resto reverso sobre el carton compartido con la obra no inv 331 sello humedo castellano 1982 ca museo sorolla 330 sello a tinta impresa azul y numero a tinta negra manual realizado tras la adjudicacion de

In [86]:
consultas_response

["SELECT COUNT(*) FROM fichas_raw WHERE coleccion ILIKE '%pintura%'",
 "SELECT * FROM fichas_raw WHERE coleccion ILIKE 'pintura' AND fecha_ano = 1905 ORDER BY inventario LIMIT 10",
 "SELECT COUNT(*) FROM fichas_raw WHERE coleccion ILIKE '%escultura%'",
 "SELECT * FROM fichas_raw WHERE (descripcion ILIKE '%marina%' OR clasificacion_razonada ILIKE '%marina%' OR historia_del_objeto ILIKE '%marina%') AND autor_a ILIKE '%sorolla%' GROUP BY inventario LIMIT 10",
 'SELECT autor_a FROM fichas_raw WHERE autor_a IS NOT NULL GROUP BY autor_a HAVING COUNT(autor_a) > 5 ORDER BY COUNT(autor_a) DESC LIMIT 10',
 'SELECT coleccion, COUNT(*) FROM fichas_raw GROUP BY coleccion',
 'SELECT * FROM fichas_raw WHERE fecha_ano IS NULL GROUP BY inventario LIMIT 10',
 'SELECT * FROM fichas_raw WHERE fecha_ano IS NOT NULL ORDER BY fecha_ano ASC LIMIT 10',
 "SELECT COUNT(*) FROM fichas_raw WHERE coleccion ILIKE 'pintura' AND tecnica ILIKE '%oleo%'",
 "SELECT * FROM fichas_raw WHERE forma_de_ingreso ILIKE '%donacio

## Evaluar los resultados

In [87]:
from ragas.metrics import DataCompyScore
from ragas.dataset_schema import SingleTurnSample
import pandas as pd

metrics_nonllm = []

for data1, data2 in zip(resultados_response, resultados_reference):
    # Si están vacíos o mal formateados, los ignoramos
    if not data1 or not data2:
        metrics_nonllm.append(float('nan'))
        continue

    # Detectar el número de columnas automáticamente
    n_cols1 = len(data1[0])
    n_cols2 = len(data2[0])

    # Crear nombres de columnas genéricos
    col_names1 = [f"col{i}" for i in range(n_cols1)]
    col_names2 = [f"col{i}" for i in range(n_cols2)]

    # Convertir a DataFrame
    df1 = pd.DataFrame(data1, columns=col_names1)
    df2 = pd.DataFrame(data2, columns=col_names2)

    # Si las columnas no coinciden, no tiene sentido comparar
    if col_names1 != col_names2:
        metrics_nonllm.append(float('nan'))
        continue

    # Convertir a CSV string
    csv1 = df1.to_csv(index=False)
    csv2 = df2.to_csv(index=False)

    # Calcular la métrica
    sample = SingleTurnSample(response=csv1, reference=csv2)
    scorer = DataCompyScore()
    metric = await scorer.single_turn_ascore(sample)
    metrics_nonllm.append(metric)

# Mostrar resultados
print("Métricas sin LLM:")
for metric in metrics_nonllm:
    print(metric)


Métricas sin LLM:
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [66]:
import numpy as np
np.mean(metrics_nonllm)

np.float64(1.0)

# Evaluación con métrica no basada en ejecución

In [96]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama(model="llama3")  # Asegúrate de tenerlo cargado localmente


In [ ]:
def evaluate_sql_equivalence(reference, response):
    prompt = f"""Te doy dos consultas SQL y un esquema de base de datos. ¿Responden a la misma intención?

    Schema:
    inventario               | text         | not null | 
    coleccion                | text (toma valores concretos de: mobiliario, cartas, escultura, textiles, pintura, fotografia, dibujo, joyeria, ceramica)
    nombre_especifico        | text 
    tipologia_estado         | text    
    lugar_de_produccion_ceca | text    
    inscripciones_leyendas   | text  
    historia_del_objeto      | text   
    iconografia              | text
    dimensiones              | text  
    contexto_cultural_estilo | text    
    descripcion              | text    
    forma_de_ingreso         | text    
    clasificacion_razonada   | text    
    componentes              | text    
    tecnica                  | text     
    clasificacion_generica   | text    
    autor_a                  | text   
    datacion                 | text    
    conjunto                 | text    
    lugar_de_procedencia     | text    
    objeto_documento         | text    
    materia_soporte          | text    
    titulo                   | text    
    imagenes                 | text    
    firmas_marcas_etiquetas  | text   
    bibliografia             | text    
    fecha_ano                | integer 
    Indexes:
        "fichas_raw_pkey" PRIMARY KEY, btree (inventario)

    Consulta de referencia:
    {reference}

    Consulta generada:
    {response}

    Responde con 1 si son equivalentes y 0 si no lo son. Empieza con "Respondo: 0/1". Explica tu razonamiento después.
    """

    output = llm.invoke(prompt).content
    return 1 if '1' in output.str.startswith('Respondo: 1') else 0, output

In [113]:
metrics_llm = []
metrics_llm_output = []

for response, reference in zip(consultas_response, consultas_reference):
    metric, output = evaluate_sql_equivalence(response, reference)
    print(response), print(reference)
    metrics_llm.append(metric)
    print(output)
    print('-'*60)
    metrics_llm_output.append(output)



SELECT COUNT(*) FROM fichas_raw WHERE coleccion ILIKE '%pintura%'
SELECT COUNT(*) FROM fichas_raw WHERE coleccion ILIKE '%pintura%'
Respondo: 1

Mi razonamiento es que tanto la consulta de referencia como la consulta generada están buscando el conteo de filas en la tabla `fichas_raw` donde el campo `coleccion` contiene la palabra "pintura" (con mayúsculas y minúsculas), utilizando la función ILIKE para hacer una búsqueda case-insensitive.

La única diferencia entre las dos consultas es que la consulta de referencia no tiene un nombre específico, mientras que la consulta generada tiene el mismo nombre (`SELECT COUNT(*) FROM fichas_raw WHERE coleccion ILIKE '%pintura%'`). Sin embargo, esto no afecta el resultado de la consulta, ya que el nombre de una consulta en SQL solo se utiliza para identificarla y no influye en su ejecución.

En resumen, tanto la consulta de referencia como la consulta generada tienen el mismo propósito y efecto, lo que hace que sean equivalentes.
-----------------

In [173]:
def evaluate_sql_equivalence(reference: str, response: str) -> tuple[int, str]:
    """
    Evaluates if two SQL queries are equivalent in their intent given a database schema.

    Args:
        reference: The reference SQL query.
        response: The generated SQL query to compare.

    Returns:
        A tuple containing:
        - An integer (1 if equivalent, 0 if not).
        - A string with the detailed reasoning from the LLM.
    """

    schema = """
    inventario               | text         | not null |
    coleccion                | text (toma valores concretos de: mobiliario, cartas, escultura, textiles, pintura, fotografia, dibujo, joyeria, ceramica)
    nombre_especifico        | text
    tipologia_estado         | text
    lugar_de_produccion_ceca | text
    inscripciones_leyendas   | text
    historia_del_objeto      | text
    iconografia              | text
    dimensiones              | text
    contexto_cultural_estilo | text
    descripcion              | text
    forma_de_ingreso         | text
    clasificacion_razonada   | text
    componentes              | text
    tecnica                  | text
    clasificacion_generica   | text
    autor_a                  | text
    datacion                 | text
    conjunto                 | text
    lugar_de_procedencia     | text
    objeto_documento         | text
    materia_soporte          | text
    titulo                   | text
    imagenes                 | text
    firmas_marcas_etiquetas  | text
    bibliografia             | text
    fecha_ano                | integer
    Indexes:
        "fichas_raw_pkey" PRIMARY KEY, btree (inventario)
    """

    prompt = f"""Dado el siguiente esquema de base de datos, determina si las dos consultas SQL proporcionadas responden a la misma intención.

            ### Esquema de la Base de Datos:
            {schema}

            ### Consulta de Referencia:
            ```sql
            {reference}
            ```

            ### Consulta Generada:
            ```sql
            {response}
            ```

            Responde **solo con el número 1 o 0** en la primera línea. Si son equivalentes, responde 1. Si no lo son, responde 0. Después, en las líneas siguientes, proporciona una explicación clara y concisa de tu razonamiento.
            """
    try:
        output = llm.invoke(prompt).content.strip() # Strip to handle any leading/trailing whitespace
        # Split the output into lines and try to get the first character of the first line
        first_line = output.split('\n')[0]
        
        # Check if the first character is '1' or '0'
        if first_line.strip() == '1':
            is_equivalent = 1
        elif first_line.strip() == '0':
            is_equivalent = 0
        else:
            # If the first line is not '0' or '1', assume non-equivalence and log it
            print(f"Warning: LLM did not return 0 or 1 plain text on the first line. Output: {output[:50]}...")
            is_equivalent = 0 # Default to 0 if expected format isn't met
            
        return is_equivalent, output
    except Exception as e:
        print(f"An error occurred during LLM invocation: {e}")
        return 0, f"Error during evaluation: {e}"

In [129]:
import pandas as pd

df = pd.DataFrame({'preguntas':preguntas, 'consultas_reference':consultas_reference, 'consultas_response':consultas_response,'resultados_reference':resultados_reference, 'resultados_response':resultados_response })
df.head()

,preguntas,consultas_reference,consultas_response,resultados_reference,resultados_response
0,¿Cuántos cuadros hay en la colección?,SELECT COUNT(*) FROM fichas_raw WHERE coleccio...,SELECT COUNT(*) FROM fichas_raw WHERE coleccio...,"[(1358,)]","[(1358,)]"
1,¿Qué pinturas fueron creadas en 1905?,SELECT * FROM fichas_raw WHERE coleccion ILIKE...,SELECT * FROM fichas_raw WHERE coleccion ILIKE...,"[(00330, pintura, None, None, javea, reverso s...","[(00330, pintura, None, None, javea, reverso s..."
2,¿Cuántas esculturas hay en el museo?,SELECT COUNT(*) FROM fichas_raw WHERE coleccio...,SELECT COUNT(*) FROM fichas_raw WHERE coleccio...,"[(229,)]","[(229,)]"
3,Dame las obras de Sorolla con temática marina.,SELECT * FROM fichas_raw WHERE autor_a ILIKE '...,SELECT * FROM fichas_raw WHERE (descripcion IL...,"[(00636, pintura, None, None, asturias, revers...","[(00636, pintura, None, None, asturias, revers..."
4,¿Qué autores tienen más de 5 obras en la colec...,"SELECT autor_a, COUNT(*) FROM fichas_raw GROUP...",SELECT autor_a FROM fichas_raw WHERE autor_a I...,[(barrera gomez juan lugar de nacimiento sevil...,[(barrera gomez juan lugar de nacimiento sevil...


In [136]:
for i, row in df.iterrows():
    respuesta = llamar_llm_respuesta(row['preguntas'], row['consultas_response'], row['resultados_response'])
    df.at[i,'respuesta'] = respuesta

RESULTADOS: [(1358,)]
RESULTADOS: [('00330', 'pintura', None, None, 'javea', 'reverso sobre el carton compartido con la obra no inv 331 tinta impresa azul 1929 ca 73 n hace referencia a la serie y numero de la testamentaria de 1929 reverso sobre el carton compartido con la obra no inv 331 escritura manual castellano 1929 ca vda de s realizado a tinta negra completa la inscripcion a tinta impresa azul con la serie y numero de la testamentaria de 1929 indicando la adjudicacion a clotilde garcia del castillo reverso etiqueta blanca sobre el carton compartido con la obra no inv 331 tinta impresa castellanoserie n 73 realizado a tinta negra la cifra escrita entre parentesis esta escrita con tinta impresa y no manual como el resto reverso sobre el carton compartido con la obra no inv 331 sello humedo castellano 1982 ca museo sorolla 330 sello a tinta impresa azul y numero a tinta negra manual realizado tras la adjudicacion del numero de inventario del museo seguramente por florencio de santa

In [145]:
df['resultados_reference']

0                                             [(1358,)]
1     [(00330, pintura, None, None, javea, reverso s...
2                                              [(229,)]
3     [(00636, pintura, None, None, asturias, revers...
4     [(barrera gomez juan lugar de nacimiento sevil...
5     [(mobiliario, 183), (cartas, 5226), (escultura...
6     [(01312, pintura, None, None, None, None, None...
7     [(20133, escultura, None, None, al andalus, No...
8                                             [(1261,)]
9     [(01345, pintura, None, None, madrid, reverso ...
10    [(14450, dibujo, None, original, None, None, N...
11    [(80207, fotografia, None, None, None, None, N...
12    [(00297, pintura, None, None, valencia, revers...
13    [(00308, pintura, None, None, comunidad valenc...
14    [(CS0167, cartas, None, None, valencia, None, ...
Name: resultados_reference, dtype: object

In [146]:
from ragas import EvaluationDataset, evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, AnswerRelevancy, FactualCorrectness
from langchain_community.chat_models import ChatOllama 
from ragas.llms import LangchainLLMWrapper
from langchain_community.chat_models import ChatOllama
from langchain.embeddings import HuggingFaceEmbeddings
import pandas as pd

llm = ChatOllama(model="llama3")  
llm_wrapper = LangchainLLMWrapper(llm)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")
dataset = []
for i, row in df.iterrows():
    # Asegura que los resultados sean una lista de strings
    raw_contexts = row['resultados_response']
    
    # Asegura que siempre sea lista
    if not isinstance(raw_contexts, list):
        raw_contexts = [raw_contexts]

    # Convierte todos los elementos a string
    contexts = [str(ctx) for ctx in raw_contexts]

    entry = {
        "user_input": str(row['preguntas']),
        "retrieved_contexts": contexts,
        "response": str(row['respuesta']),
    }
    dataset.append(entry)

my_eval_dataset = EvaluationDataset.from_list(dataset)


from ragas.metrics import AnswerRelevancy, ContextRelevance, Faithfulness, ResponseGroundedness


resultados = evaluate(
    dataset=my_eval_dataset,
    metrics=[
        AnswerRelevancy(), ContextRelevance(), Faithfulness(), ResponseGroundedness()
        ],
    llm=llm_wrapper,
    embeddings=embeddings 
)

print(resultados)

Evaluating:  17%|█▋        | 10/60 [03:00<21:51, 26.23s/it]Exception raised in Job[5]: TimeoutError()
Exception raised in Job[6]: TimeoutError()
Exception raised in Job[7]: TimeoutError()
Exception raised in Job[10]: TimeoutError()
Exception raised in Job[13]: TimeoutError()
Exception raised in Job[14]: TimeoutError()
Exception raised in Job[15]: TimeoutError()
Evaluating:  48%|████▊     | 29/60 [06:00<09:14, 17.88s/it]Exception raised in Job[26]: TimeoutError()
Exception raised in Job[27]: TimeoutError()
Exception raised in Job[29]: TimeoutError()
Exception raised in Job[30]: TimeoutError()
Exception raised in Job[31]: TimeoutError()
Exception raised in Job[33]: TimeoutError()
Evaluating:  78%|███████▊  | 47/60 [08:41<02:16, 10.52s/it]Exception raised in Job[45]: TimeoutError()
Exception raised in Job[46]: TimeoutError()
Evaluating:  80%|████████  | 48/60 [09:00<02:36, 13.04s/it]Exception raised in Job[47]: TimeoutError()
Exception raised in Job[49]: TimeoutError()
Exception raised in

{'answer_relevancy': 0.2754, 'nv_context_relevance': 0.5000, 'faithfulness': nan, 'nv_response_groundedness': 1.0000}


In [148]:
resultados.to_pandas()

,user_input,retrieved_contexts,response,answer_relevancy,nv_context_relevance,faithfulness,nv_response_groundedness
0,¿Cuántos cuadros hay en la colección?,"[(1358,)]","En la colección del Museo Sorolla, hay un tota...",0.151967,0.0,NaN,1.0
1,¿Qué pinturas fueron creadas en 1905?,"[('00330', 'pintura', None, None, 'javea', 're...","En 1905, Sorolla creó varias pinturas notables...",0.359539,NaN,NaN,NaN
2,¿Cuántas esculturas hay en el museo?,"[(229,)]","En el Museo Sorolla, tenemos un total de 229 e...",0.444583,1.0,NaN,1.0
3,Dame las obras de Sorolla con temática marina.,"[('00636', 'pintura', None, None, 'asturias', ...",Entre las obras de Sorolla con temática marina...,0.416601,NaN,NaN,NaN
4,¿Qué autores tienen más de 5 obras en la colec...,[('barrera gomez juan lugar de nacimiento sevi...,"En nuestra colección, hay varios autores con m...",0.067723,0.5,NaN,NaN
5,¿Cuántas obras hay por colección?,"[('mobiliario', 183), ('cartas', 5226), ('escu...","En el Museo Sorolla, contamos con una variedad...",0.146893,NaN,NaN,NaN
6,¿Qué obras están sin fecha conocida?,"[('01312', 'pintura', None, None, None, None, ...","En el Museo Sorolla, hay varias obras cuya fec...",0.093738,NaN,NaN,NaN
7,¿Cuáles son las obras más antiguas del museo?,"[('20133', 'escultura', None, None, 'al andalu...","Entre las obras más antiguas del museo, destac...",0.358948,NaN,NaN,NaN
8,¿Cuántas pinturas hay con técnica al óleo?,"[(1261,)]","En el Museo Sorolla, contamos con una gran var...",0.215450,NaN,NaN,1.0
9,¿Qué obras fueron donadas?,"[('01345', 'pintura', None, None, 'madrid', 'r...",Entre las obras donadas al museo se encuentran...,0.147114,NaN,NaN,NaN


## Evaluación sin instrucciones previas

In [ ]:
def llamar_llm_sql(client, llm_modelname, mensaje, contexto=""):
    prompt_sql = f"""
        Genera una consulta SQL para responder la siguiente pregunta del usuario, usando la tabla `fichas_raw`, que tiene las siguientes columnas:
        - inventario
        - coleccion (mobiliario, cartas, escultura, textiles, pintura, fotografia, dibujo, joyeria, ceramica)
        - contexto_cultural_estilo
        - dimensiones
        - iconografia
        - historia_del_objeto
        - lugar_de_produccion_ceca
        - componentes
        - tecnica
        - conjunto
        - titulo
        - autor_a
        - bibliografia
        - descripcion
        - lugar_de_procedencia
        - nombre_especifico
        - clasificacion_razonada
        - materia_soporte
        - imagenes
        - forma_de_ingreso
        - firmas_marcas_etiquetas
        - datacion (datación aproximada)
        - fecha_ano (año de datación)
        - inscripciones_leyendas
        - objeto_documento
        - clasificacion_generica

        Pregunta del usuario: {mensaje}

        Devuelve únicamente el texto de la consulta SQL sin comentarios ni formato adicional ni ```sql.
        """


    completion = client.chat.completions.create(
        model=llm_modelname,
        messages=[{"role": "user", "content": prompt_sql}],
        temperature=0.0,
        max_completion_tokens=512,
        top_p=1,
        stream=True
    )

    # Obtener la respuesta de la consulta SQL y comprobar que es válida (solo SELECT)   
    sql_respuesta = ""
    for chunk in completion:
        sql_respuesta += chunk.choices[0].delta.content or ""

    if not sql_respuesta.lower().startswith("select"):
        raise ValueError("La consulta SQL generada no es válida. Debe comenzar con 'SELECT'.")

    return sql_respuesta


    from groq import Groq

# Configuración de conexión (ajusta esto a tus credenciales)
conn = psycopg2.connect(
    host="localhost",
    database="museo",
    user="fedecl",
    password="password",
    port=5432 
)

client = Groq(api_key) #completar

consultas_response_sin_instr = []


for pregunta in preguntas:
    consulta = llamar_llm_sql(client, "mistral-saba-24b", pregunta)
    consultas_response_sin_instr.append(consulta)

cur = conn.cursor()

resultados_response_sin_instr = []

for consulta in consultas_reference:
    cur.execute(consulta)
    resultado = cur.fetchall()
    resultados_response_sin_instr.append(resultado)

# Cerrar conexión
cur.close()
conn.close()
    

In [161]:
from ragas.metrics import DataCompyScore
from ragas.dataset_schema import SingleTurnSample
import pandas as pd

metrics_nonllm_sin_instr = []

for data1, data2 in zip(resultados_response_sin_instr, resultados_reference):
    # Si están vacíos o mal formateados, los ignoramos
    if not data1 or not data2:
        metrics_nonllm_sin_instr.append(float('nan'))
        continue

    # Detectar el número de columnas automáticamente
    n_cols1 = len(data1[0])
    n_cols2 = len(data2[0])

    # Crear nombres de columnas genéricos
    col_names1 = [f"col{i}" for i in range(n_cols1)]
    col_names2 = [f"col{i}" for i in range(n_cols2)]

    # Convertir a DataFrame
    df1 = pd.DataFrame(data1, columns=col_names1)
    df2 = pd.DataFrame(data2, columns=col_names2)

    # Si las columnas no coinciden, no tiene sentido comparar
    if col_names1 != col_names2:
        metrics_nonllm_sin_instr.append(float('nan'))
        continue

    # Convertir a CSV string
    csv1 = df1.to_csv(index=False)
    csv2 = df2.to_csv(index=False)

    # Calcular la métrica
    sample = SingleTurnSample(response=csv1, reference=csv2)
    scorer = DataCompyScore()
    metric = await scorer.single_turn_ascore(sample)
    metrics_nonllm_sin_instr.append(metric)

# Mostrar resultados
print("Métricas sin LLM:")
for metric in metrics_nonllm_sin_instr:
    print(metric)


Métricas sin LLM:
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [174]:
metrics_llm_sin_instr = []
metrics_llm_output_sin_instr = []

for response, reference in zip(consultas_response_sin_instr, consultas_reference):
    metric, output = evaluate_sql_equivalence(response, reference)
    print(response), print(reference)
    metrics_llm_sin_instr.append(metric)
    print(output)
    print('-'*60)
    metrics_llm_output_sin_instr.append(output)


Las dos consultas SQL tienen la misma inten...
SELECT COUNT(*) FROM fichas_raw WHERE coleccion = 'pintura';
SELECT COUNT(*) FROM fichas_raw WHERE coleccion ILIKE '%pintura%'
**1**

Las dos consultas SQL tienen la misma intención.

La razón es que tanto la consulta de referencia como la consulta generada buscan contar el número de registros en la tabla `fichas_raw` donde la columna `coleccion` contiene el valor `'pintura'`. La única diferencia entre las dos consultas es que la consulta generada utiliza el operador ILIKE (case-insensitive LIKE) y un wildcard (`%`) para buscar cualquier cadena que contenga el texto `'pintura'`, mientras que la consulta de referencia utiliza el operador igual (`=`) y no utiliza wildcards.

Sin embargo, en este caso, ya que se está buscando exactamente el valor `'pintura'` (sin considerar mayúsculas o minúsculas), tanto la consulta de referencia como la consulta generada devolverán el mismo resultado.
-------------------------------------------------------

In [175]:
metrics_llm_sin_instr

[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]